In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import seaborn as sns
from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
#### read in the flattened dataframes with the labels
MSFT_df = pd.read_csv("/Users/fahad/project_repo/data/external/MSFT_flat.csv", index_col=0)
AAPL_df = pd.read_csv("/Users/fahad/project_repo/data/external/AAPL_flat.csv",index_col=0)
AMZN_df = pd.read_csv("/Users/fahad/project_repo/data/external/AMZN_flat.csv",index_col=0)

In [3]:
MSFT_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,351,352,353,354,355,356,357,358,359,Label
1998-03-31,0.429339,0.425285,0.432434,0.426907,0.020233,0.627285,0.373432,0.500535,0.519246,0.231693,...,0.703805,0.005767,0.226505,0.019309,0.457683,0.712130,0.453455,0.0,0.0,1
1998-04-30,0.444974,0.443069,0.449092,0.441714,0.005799,0.403578,0.305363,0.475177,0.588950,0.276414,...,0.844894,0.012340,0.954846,0.966738,0.509452,0.514381,0.640977,0.0,0.0,1


In [4]:

#vertically stack the MSFT and AAPL dataframes on top of eachother
df = pd.concat([MSFT_df, AAPL_df], axis=0)
#vertically stack the MSFT&AAPL df and the AMZN dataframe ontop of eachother
df = pd.concat([df,AMZN_df],axis = 0)
#set index to date
#df.set_index("Date")

y = df.Label # define the target variable (dependent variable) as y

df = df.drop(['Label'], axis =1)


In [5]:
#define the target variable as y 
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3)
#print (X_train.head(5))
print(X_train.shape[1])
print(X_train.shape[0])

360
537


# Model

In [6]:
def make_timeseries_classifier(sample_num,window_size, filter_length, nb_input_series=1, nb_outputs=1, nb_filter=16):
    """:Return: a Keras Model for predicting the next value in a timeseries given a fixed-size lookback window of previous values.
    The model can handle multiple input timeseries (`nb_input_series`) and multiple prediction targets (`nb_outputs`).
    :param int window_size: The number of previous timeseries values to use as input features.  Also called lag or lookback.
    :param int nb_input_series: The number of input timeseries; 1 for a single timeseries.
      The `X` input to ``fit()`` should be an array of shape ``(n_instances, window_size, nb_input_series)``; each instance is
      a 2D array of shape ``(window_size, nb_input_series)``.  For example, for `window_size` = 3 and `nb_input_series` = 1 (a
      single timeseries), one instance could be ``[[0], [1], [2]]``. See ``make_timeseries_instances()``.
    :param int nb_outputs: The output dimension, often equal to the number of inputs.
      For each input instance (array with shape ``(window_size, nb_input_series)``), the output is a vector of size `nb_outputs`,
      usually the value(s) predicted to come after the last value in that input instance, i.e., the next value
      in the sequence. The `y` input to ``fit()`` should be an array of shape ``(n_instances, nb_outputs)``.
    :param int filter_length: the size (along the `window_size` dimension) of the sliding window that gets convolved with
      each position along each instance. The difference between 1D and 2D convolution is that a 1D filter's "height" is fixed
      to the number of input timeseries (its "width" being `filter_length`), and it can only slide along the window
      dimension.  This is useful as generally the input timeseries have no spatial/ordinal relationship, so it's not
      meaningful to look for patterns that are invariant with respect to subsets of the timeseries.
    :param int nb_filter: The number of different filters to learn (roughly, input patterns to recognize).
    """
    model = Sequential((
        # The first conv layer learns `nb_filter` filters (aka kernels), each of size ``(filter_length, nb_input_series)``.
        # Its output will have shape (None, window_size - filter_length + 1, nb_filter), i.e., for each position in
        # the input timeseries, the activation of each filter at that position.
        Convolution1D(nb_filter=nb_filter, filter_length=filter_length, activation='relu', input_shape=(window_size,nb_input_series)),
        MaxPooling1D(),     # Downsample the output of convolution by 2X.
        Convolution1D(nb_filter=nb_filter, filter_length=filter_length, activation='relu'),
        MaxPooling1D(),
        Flatten(),
        Dense(nb_outputs, activation='sigmoid'),     # For binary classification, change the activation to 'sigmoid'
    ))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # To perform (binary) classification instead:
    # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    
    return model

In [7]:

model_1 = make_timeseries_classifier(X_train.shape[0],X_train.shape[1],3)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", input_shape=(360, 1), filters=16, kernel_size=3)`
W0717 16:36:03.042312 4589483456 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=16, kernel_size=3)`
W0717 16:36:03.059471 4589483456 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 16:36:03.061857 4589483456 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: 

In [8]:
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 358, 16)           64        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 179, 16)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 177, 16)           784       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 88, 16)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1408)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1409      
Total params: 2,257
Trainable params: 2,257
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history=model_1.fit(np.array(X_train).reshape((*X_train.shape,1)),np.array(y_train).reshape((*y_train.shape,1)),
            validation_split=0.10, epochs=100
            ,batch_size=1, verbose=1)

W0717 16:36:03.386466 4589483456 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 483 samples, validate on 54 samples
Epoch 1/100
483/483 [==============================] - 1s 2ms/step - loss: 0.6899 - acc: 0.5714 - val_loss: 0.6567 - val_acc: 0.5926
Epoch 2/100
483/483 [==============================] - 1s 1ms/step - loss: 0.6543 - acc: 0.6232 - val_loss: 0.6258 - val_acc: 0.6111
Epoch 3/100
483/483 [==============================] - 1s 1ms/step - loss: 0.5968 - acc: 0.6729 - val_loss: 0.5273 - val_acc: 0.6852
Epoch 4/100
483/483 [==============================] - 1s 1ms/step - loss: 0.5404 - acc: 0.7288 - val_loss: 0.5326 - val_acc: 0.7037
Epoch 5/100
483/483 [==============================] - 1s 1ms/step - loss: 0.5215 - acc: 0.7412 - val_loss: 0.4708 - val_acc: 0.7037
Epoch 6/100
483/483 [==============================] - 1s 1ms/step - loss: 0.4881 - acc: 0.7764 - val_loss: 0.4760 - val_acc: 0.7037
Epoch 7/100
483/483 [==============================] - 1s 1ms/step - loss: 0.4607 - acc: 0.7930 - val_loss: 0.5117 - val_acc: 0.7407
Epoch 8/100
483/483 [===

# Predict The Test Data

In [ ]:
#make the predictions using the the model we have created, note the input must be a np.array and a 3d tensor
predictions = model_1.predict_classes(np.array(X_test).reshape((*X_test.shape,1)),batch_size = X_test.shape[0], verbose=1)

In [ ]:
predictions

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

In [ ]:
#plot the accuracy of the training set  and the validation set of the model
import matplotlib.pyplot as plt
plt.figure(figsize=[20,20])
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylabel(['Accuracy'])
plt.xlabel(['Epochs'])
plt.show()


In [ ]:

#import h5py

#stopping callback to get the validation 
#keras.callbacks.EarlyStopping(monitor_metric='val accuracy', mode='max', patience='callback_patience')

#saving criteria
#keras.callbacks.ModelCheckpoint(model=name of model.h5, 
#monitor=monitor_metric = 'val_accuracy' mode='max',savE_best_only=True)

#in model.fit(.........callbacks=callback_list )
